<a href="https://colab.research.google.com/github/FirmanAhmadBayumi/Deep-Learning/blob/main/PembagianDataSet_Augmentasi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import cv2
import numpy as np
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm

In [3]:
# Define the base directory where all images are stored
BASE_DIR = '/content/drive/MyDrive/Computer Vision/NonAugmented'
AUG_DIR = '/content/drive/MyDrive/Computer Vision/Augmented'

train_dir = os.path.join(BASE_DIR, 'train')
val_dir = os.path.join(BASE_DIR, 'validation')
test_dir = os.path.join(BASE_DIR, 'test')

aug_train_dir = os.path.join(AUG_DIR, 'train')
aug_val_dir = os.path.join(AUG_DIR, 'validation')
aug_test_dir = os.path.join(AUG_DIR, 'test')

In [4]:
IMG_SIZE = (299, 299)

# Create directories if not exist
os.makedirs(aug_train_dir, exist_ok=True)
os.makedirs(aug_val_dir, exist_ok=True)
os.makedirs(aug_test_dir, exist_ok=True)

In [5]:
# Define data augmentation generator
data_gen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [6]:
def copy_and_augment_data(src_dir, dest_dir, img_size, data_gen, save_prefix='aug', num_augmented=2):
    class_names = os.listdir(src_dir)

    for class_name in class_names:
        class_src_dir = os.path.join(src_dir, class_name)
        class_dest_dir = os.path.join(dest_dir, class_name)
        os.makedirs(class_dest_dir, exist_ok=True)

        for file in tqdm(os.listdir(class_src_dir), desc=f"Processing {class_name}"):
            img_path = os.path.join(class_src_dir, file)
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.webp')):
                image = cv2.imread(img_path)
                if image is not None:
                    # Copy original file to destination
                    shutil.copy(img_path, class_dest_dir)

                    # Resize for augmentation
                    image = cv2.resize(image, img_size)
                    image = np.expand_dims(image, axis=0)
                    image_gen = data_gen.flow(image, batch_size=1, save_to_dir=class_dest_dir,
                                              save_prefix=save_prefix, save_format='jpg')

                    # Generate and save augmented images
                    for _ in range(num_augmented):
                        next(image_gen)

In [7]:
# Copy and augment data for train, validation, and test directories
copy_and_augment_data(train_dir, aug_train_dir, IMG_SIZE, data_gen, num_augmented=2)
copy_and_augment_data(val_dir, aug_val_dir, IMG_SIZE, data_gen, num_augmented=2)
copy_and_augment_data(test_dir, aug_test_dir, IMG_SIZE, data_gen, num_augmented=2)

print("Data copy and augmentation completed.")

Processing kupu_raja_limau: 100%|██████████| 20/20 [00:18<00:00,  1.07it/s]

Data copy and augmentation completed.
